# Gaussian Mixture Model (GMM) + Random Forest Regression (RFR) Code

## Gaussian Mixture Model

#### This GMM is fit based on the GLODAP 2022 Data. Cluster assignments for new data will need to be predicted using this GMM. 

In [ ]:
#OPTIMAL CLUSTER ALG: 8 CLUSTERS,NO LON, DEPTH as an additional predictor
#ta_ed refers to the dataframe that is being used
gmm_feat = ["Latitude", "SST", "SSS", "Bottom Depth"]
gmm = GaussianMixture(n_components = 8, covariance_type = "full", random_state = 42)
gmm.fit(ta_ed[gmm_feat])

components = gmm.predict(ta_ed[gmm_feat])
prob = gmm.predict_proba(ta_ed[gmm_feat])

In [ ]:
## Creating a cluster column that has the assigned clusters for each data point
ta_ed["Cluster"] = components
## GMM outputs a probability that each data point belongs to each cluster; this code saves that information as a dataframe
prob_df = pd.DataFrame(prob, columns = ["Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", "Cluster 5", "Cluster 6", "Cluster 7", "Cluster 8"])

In [ ]:
## Creating new dataframes for each cluster
clust1 = ta_ed.loc[ta_ed["Cluster"] ==0]
clust2 = ta_ed.loc[ta_ed["Cluster"] ==1]
clust3 = ta_ed.loc[ta_ed["Cluster"] ==2]
clust4 = ta_ed.loc[ta_ed["Cluster"] ==3]
clust5 = ta_ed.loc[ta_ed["Cluster"] ==4]
clust6 = ta_ed.loc[ta_ed["Cluster"] ==5]
clust7 = ta_ed.loc[ta_ed["Cluster"] ==6]
clust8 = ta_ed.loc[ta_ed["Cluster"] ==7]

#Create a list of dataframes; this is the input for the RFR code
cluster_lst = [clust1, clust2, clust3, clust4, clust5, clust6, clust7, clust8]

## Random Forest Regression

### Preparing the Data

In [ ]:
# This code iterates through a list of clusters and thus does not need to be individually run on each cluster

# The clusters (e.g. clust1) should be dataframes and the dataframes should be stored in a list
cluster_lst = [clust1, clust2, clust3, clust4, clust5, clust6, clust7, clust8]

#MAKING FEATURES: make features specific for each cluster
##Making a dictionary where the keys represent clusters
def make_features(c_lst):
    ft_dict = {}
    for idx, cluster in enumerate(c_lst):
        X1 = cluster["S"] #0
        X2 = cluster["PT"] #1
        X3 = cluster["Nitrate"] #2
        X4 = cluster["AOU"] #3
        X5 = cluster["Silicate"] #4
        lon20= np.cos(np.deg2rad(cluster["Longitude"] - 20))#5
        lon110= np.cos(np.deg2rad(cluster["Longitude"] - 110)) #6
        depth = cluster["Depth"] #7
        order = cluster["order"] #8
        y = cluster["TA"] #9
        cruise = cluster["Cruise"] #10
        ft_dict[idx]=[X1, X2, X3, X4, X5, lon20, lon110, depth, order, y, cruise]
    return ft_dict

#TRAIN-TEST SPLIT: split the data for each cluster
##Don't want to split cruises between the training and testing groups
def split_data(ft_dict):
    split_dict = {}
    for idx in ft_dict.keys(): #this is going thru each cluster
        features = ft_dict[idx]
        X, y, cruise = features[0], features[8], features[9]
        clustersp = GroupShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 40) 
        ind = list(clustersp.split(X, y, cruise))
        train_ind, test_ind = ind[0][0], ind[0][1]
        split_dict[idx] = [train_ind, test_ind]
    return split_dict

#RESHAPED DATA: Reshape the data so it is organized in a way that the random forest can take the data
##Drop y, order, and cruise columns here
def rearrange(splitted_data, ft_dict):
    reshaped_data = {}
    for idx in splitted_data.keys(): #going into each of the keys/clusters
        train_index, test_index = splitted_data[idx]
        feat_values = []
        for i in range(len(ft_dict[idx])-3): ##added a "-3" to get rid of last three elements (which are the y, order, and cruise columns)
            re_values = []
            train = np.array(ft_dict[idx][i])[train_index]
            train.reshape(-1,1)
            test = np.array(ft_dict[idx][i])[test_index]
            test.reshape(-1,1)
            re_values.append(train)
            re_values.append(test)
            feat_values.append(re_values)
        reshaped_data[idx] = feat_values
    return reshaped_data

#TA (Y) VALUES: Making a dictionary of all the y-values for the training and testing data
##Each of the keys is for a cluster
def y_capture(splitted_data, ft_dict):
    y_dict = {}
    for idx in splitted_data.keys(): #going into each of the keys 
        train_index, test_index = splitted_data[idx]
        y_train = ft_dict[idx][10].iloc[train_index]
        y_test = ft_dict[idx][10].iloc[test_index]
        y_dict[idx] = (y_train, y_test)
    return y_dict

#ORDER: Making a dictionary of all the order for the training and testing data
##Each of the keys is for a cluster
def order_capture(splitted_data, ft_dict): #this is for the order col
    order_dict = {}
    for idx in splitted_data.keys(): #going into each of the keys 
        train_index, test_index = splitted_data[idx]
        order_train = ft_dict[idx][8].iloc[train_index]
        order_test = ft_dict[idx][8].iloc[test_index]
        order_dict[idx] = (order_train, order_test)
    return order_dict

#REARRANGING THE DATA INTO THE ESPER EQUATION 1: making sure the data is in a format 
def make_clust_eq(reshaped_dict):
    tpose_dt = {}
    for idx in reshaped_dict.keys(): #going into each cluster
        final_set = []
        train_eq_clust = []
        test_eq_clust = []
        for k in range(len(reshaped_dict[idx])):
            train_eq_clust.append(reshaped_dict[idx][k][0]) #this is where the problem is 
            train_clust_trans = np.transpose(train_eq_clust)
            test_eq_clust.append(reshaped_dict[idx][k][1]) #this is where the problem is 
            test_clust_trans = np.transpose(test_eq_clust)
        final_set.append(train_clust_trans)
        final_set.append(test_clust_trans)
        tpose_dt[idx] = final_set
    return tpose_dt

### Running the Random Forest Model

In [ ]:
#RMSE DICTIONARY: run the random forest model to get the training and testing RMSEs for each cluster 
def rfr(tpose_dt, y_dict):
    train_rmse_lst = []
    tst_rmse_lst = []
    for idx in tpose_dt.keys():
        rf = RandomForestRegressor()
        rf.fit(tpose_dt[idx][0], y_dict[idx][0])
        
        train_pred = rf.predict(tpose_dt[idx][0])
        train_rmse = root_mean_squared_error(y_dict[idx][0], train_pred)
        train_rmse_lst.append(train_rmse)
        
        test_pred = rf.predict(tpose_dt[idx][1])
        tst_rmse = root_mean_squared_error(y_dict[idx][1], test_pred)
        tst_rmse_lst.append(tst_rmse)
    return train_rmse_lst, tst_rmse_lst

### Improving the Model

In [ ]:
#HYPERPARAMETER TUNING
def hyp_tr(tpose_dt, y_dict):
    p_dist = {"n_estimators": randint(50, 500),
          "max_depth": randint(1,20),
          "max_features": randint(1,9)}
    hyp_dict = {}
    for idx in tpose_dt.keys(): 
        rf = RandomForestRegressor()
        rs = RandomizedSearchCV(rf, 
                        param_distributions = p_dist, 
                        n_iter = 5,
                        cv =5)
        res = rs.fit(tpose_dt[idx][0], y_dict[idx][0])
        hyp_dict[idx] = [list(res.best_params_.values())[0],list(res.best_params_.values())[1], list(res.best_params_.values())[2]]
    return hyp_dict

#MODEL BASED ON CHOSEN HYPERPARAMETERS
def best_model(tpose_dt, y_dict, hyp_dict):
    best_rmse = {}
    best_dict = {}
    for idx in tpose_dt.keys():
        best_rf = RandomForestRegressor(max_depth=hyp_dict[idx][0], n_estimators=hyp_dict[idx][1], max_features = hyp_dict[idx][2])
        best_rf.fit(tpose_dt[idx][0], y_dict[idx][0])
        best_preds = best_rf.predict(tpose_dt[idx][1])
        rmse = root_mean_squared_error(y_dict[idx][1], best_preds)
        best_rmse[idx] = rmse
        best_dict[idx] = best_preds
    return best_rmse, best_dict

### Outputting TA Predictions

In [ ]:
#PREDICTION DICTIONARY: output a dictionary of all the training and testing set predictions
def preds(tpose_dt, y_dict):
    train_preds = {}
    test_preds = {}
    for idx in tpose_dt.keys():
        rf = RandomForestRegressor()
        rf.fit(tpose_dt[idx][0], y_dict[idx][0])
        train_pred = rf.predict(tpose_dt[idx][0])
        train_preds[idx] = train_pred
        test_pred = rf.predict(tpose_dt[idx][1])
        test_preds[idx] = test_pred
    return train_preds, test_preds

#PREDICTION DATAFRAME: output a dataframe of the testing set predictions with the order #, cluster #, and actual TA value
def comb_ords_preds(test_preds, order_dict, y_dict):
    clnum = []
    ord_tst = []
    pred_tst = []
    actual_y = []
    for idx in range(len(orders)):
        cl_len = [idx]*len(orders[idx][1])
        ord_tst.extend(orders[idx][1])
        clnum.extend(cl_len)
    for idx in range(len(y_dict)):
        actual_y.extend(y_dict[idx][1])
    for i in range(len(test_preds)):
        pred_tst.extend(test_preds[i])
    comb_df = pd.DataFrame( {"Order_Num": ord_tst,
                             "RFR1": pred_tst,
                             "Y_test": actual_y,
                             "Cluster": clnum})
    return comb_df